<a href="https://colab.research.google.com/github/KendraR9/Reto-MeIA/blob/main/reto_clasificacion_especies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Desafío de clasificación de especies

## Objetivo:
Encontrar el método de aprendizaje automático supervisado que muestra mayor eficacia para clasificar especies a partir de secuencias de código de barras de ADN.

## Propuesta:
Se cuenta con un método disponible (randomforest classifier) cuya eficacia ya es muy buena, por lo que es posible realizar un tratamiento de los datos antes de utilizar el método para:
- Disminuir el consumo de recursos computacionales
- Mejorar la eficacia del método con el que se cuenta
- Automatizar en mayor medida posible el flujo de trabajo

Comenzamos instalando Biopython y las librerias necesarias para realizar el tratamiento de datos:

In [1]:
!pip install biopython

In [2]:
import Bio
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio import Align
from Bio import AlignIO

Importamos los archivos FASTA con los conjuntos de entrenamiento y prueba:

In [3]:
from google.colab import files
upload = files.upload()

Saving Drosophila_train.fasta to Drosophila_train (1).fasta
Saving Drosophila_test.fasta to Drosophila_test (1).fasta


# 2. Conjunto de prueba

Se buscó automatizar la lectura de los archivos en formato FASTA, anexar secuencias auxiliares que serán necesarias al momento de realizar _one-hot encoding_. Así mismo, se realizó un alineamiento múltiple de las secuencias y se generó el archivo .aln que puede ser abierto en cualquier programa para realizar alineamientos. Esto permite ubicar los sitios conservados a lo largo de las secuencias.

Para esto, se trabajó principalmente en Biopython y sus respectivos módulos.

In [4]:
"""
Preparando secuencias para alineamiento

"""
# Generando listas vacías para los identificadores y las secuencias a insertar

idt = []
seqs = []
name = []
for record in SeqIO.parse('Drosophila_test.fasta', "fasta"):
    idt.append(record.id.split('|')[2])
    seqs.append(record.seq)
    name.append(record.name)

"""
Escribimos una funcion que lee la primera secuencia de nuestra lista
de secuencias.

Tomando la longitud de esta secuencia, podemos escribir las secuencias
auxiliares.

"""

def auxiliar(s, i, nam):
    first_record = s[0]

    a = ['A' for i in range(len(first_record))]
    a = ''.join(a)
    a_r = SeqRecord(a)
    a_r.id = 'Auxiliar'
    a_r.name = 'Auxiliar'

    c = ['C' for i in range(len(first_record))]
    c = ''.join(c)
    c_r = SeqRecord(c)
    c_r.id = 'Auxiliar'
    c_r.name = 'Auxiliar'

    t = ['T' for i in range(len(first_record))]
    t = ''.join(t)
    t_r = SeqRecord(t)
    t_r.id = 'Auxiliar'
    t_r.name = 'Auxiliar'

    g = ['G' for i in range(len(first_record))]
    g = ''.join(g)
    g_r = SeqRecord(g)
    g_r.id = 'Auxiliar'
    g_r.name = 'Auxiliar'

    n = ['N' for i in range(len(first_record))]
    n = ''.join(n)
    n_r = SeqRecord(n)
    n_r.id = 'Auxiliar'
    n_r.name = 'Auxiliar'

    print
    aux = [a_r, c_r, t_r, g_r, n_r]

    for record in aux:
        i.append(record.id)
        s.append(record.seq)
        nam.append(record.name)

auxiliar(seqs, idt, name) #agrego 5 secuencias auxiliares para el one-hot encoding

In [5]:
sequences = []
with open("pre-align.fasta", "w") as output_handle:
    for i in range(len(seqs)):
        sequences = SeqRecord(Seq(seqs[i]), id =idt[i])
        SeqIO.write(sequences, output_handle, "fasta") #almacenando el archivo .fasta con secuencias auxiliares

In [6]:
"""
Alineamiento múltiple de secuencias

"""

from Bio.Align import MultipleSeqAlignment

with open("align.aln", "w") as output_handle:
    msa = MultipleSeqAlignment(SeqIO.parse("pre-align.fasta", "fasta"))
    SeqIO.write(msa, output_handle, "fasta") #almacenando el archivo del alineamiento

In [7]:
"""
Creación del dataframe del conjunto de prueba

"""

align = AlignIO.read("align.aln", "fasta")

seqs = []
idt = []

for record in align:
    seqs.append(record.seq)
    idt.append(record.id)

#Creamos un dataframe solo con las secuencias y agregamos un identificador para cada nucleotido
df_align = pd.DataFrame(align)
length = len(seqs[0]) + 1
r = list(range(1,length))
row = ['S' + str(i) for i in r]
df_align = df_align.set_axis(row, axis='columns')

df_align.insert(0, 'ID', idt)
df_align

#Reemplazamos los espacios blancos (gaps) por N
data = df_align.replace(r'-', 'N', regex=True)
data = data.copy()
print(data)

                     ID S1 S2 S3 S4 S5 S6 S7 S8 S9  ... S654 S655 S656 S657  \
0    DQ471543_110189412  A  T  T  G  G  A  A  C  T  ...    A    C    A    T   
1    DQ471554_110189434  A  T  T  G  G  A  A  C  T  ...    A    C    A    T   
2     DQ383677_87475153  N  N  N  N  N  N  A  C  T  ...    A    C    A    T   
3     DQ383678_87475155  N  N  N  N  N  N  A  C  T  ...    A    C    A    T   
4     DQ383671_87475141  N  N  N  N  N  N  A  C  T  ...    A    C    A    T   
..                  ... .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...   
116            Auxiliar  A  A  A  A  A  A  A  A  A  ...    A    A    A    A   
117            Auxiliar  C  C  C  C  C  C  C  C  C  ...    C    C    C    C   
118            Auxiliar  T  T  T  T  T  T  T  T  T  ...    T    T    T    T   
119            Auxiliar  G  G  G  G  G  G  G  G  G  ...    G    G    G    G   
120            Auxiliar  N  N  N  N  N  N  N  N  N  ...    N    N    N    N   

    S658 S659 S660 S661 S662 S663  
0      T    T  

# 3. Conjunto de entrenamiento

El conjunto de entrenamiento recibió un tratamiento similar al del conjunto de prueba. A partir de este conjunto se identificarán los sitios de homología.

In [8]:
"""
Preparando secuencias para alineamiento

"""
# Generando listas vacías para los identificadores y las secuencias a insertar

idt_train = []
seqs_train = []
name_train = []
for record in SeqIO.parse("Drosophila_train.fasta", "fasta"):
    idt_train.append(record.id)
    seqs_train.append(record.seq)

auxiliar(seqs, idt, name) #agrego 5 secuencias auxiliares

In [9]:
sequences_train = []
with open("pre-align-train.fasta", "w") as output_handle:
    for i in range(len(seqs_train)):
        sequences_train = SeqRecord(Seq(seqs_train[i]), id =idt_train[i])
        SeqIO.write(sequences_train, output_handle, "fasta")

In [10]:
from Bio.Align import MultipleSeqAlignment

with open("align-train.aln", "w") as output_handle:
    msa_train = MultipleSeqAlignment(SeqIO.parse("pre-align-train.fasta", "fasta"))
    SeqIO.write(msa_train, output_handle, "fasta")

In [11]:
"""
Creación del dataframe del conjunto de entrenamiento

"""

align_train = AlignIO.read("align-train.aln", "fasta")

seqs_train = []
idt_train = []
target = []

for record in align_train:
    seqs_train.append(record.seq)
    idt_train.append(record.id.split('|')[2])
    target.append(record.id.split('|')[1])

#Creamos un dataframe solo con las secuencias y agregamos un identificador para cada nucleotido
df_align_train = pd.DataFrame(align_train)
length = len(seqs_train[0]) + 1
r = list(range(1,length))
row = ['S' + str(i) for i in r]
df_align_train = df_align_train.set_axis(row, axis='columns')

df_align_train.insert(0, 'ID', idt_train)
df_align_train.insert(1, 'Target', target)


#Reemplazamos los espacios blancos (gaps) por N
data_train = df_align_train.replace(r'-', 'N', regex=True)
data_train = data_train.copy()
print(data_train)

                     ID              Target S1 S2 S3 S4 S5 S6 S7 S8  ... S654  \
0    DQ471538_110189402    Drosophila_angor  A  T  T  G  G  A  A  C  ...    A   
1    DQ471539_110189404    Drosophila_angor  A  T  C  G  G  A  A  C  ...    A   
2    DQ471540_110189406    Drosophila_angor  A  T  T  G  G  A  A  C  ...    A   
3    DQ471541_110189408    Drosophila_angor  A  T  C  G  G  T  A  C  ...    A   
4    DQ471542_110189410    Drosophila_angor  A  T  T  G  G  A  A  C  ...    A   
..                  ...                 ... .. .. .. .. .. .. .. ..  ...  ...   
494   DQ426805_90018989  Drosophila_virilis  N  N  N  G  G  A  A  C  ...    A   
495   DQ426806_90018991  Drosophila_virilis  N  N  N  G  G  A  A  C  ...    A   
496   DQ426807_90018993  Drosophila_virilis  N  N  N  G  G  A  A  C  ...    A   
497  DQ471535_110189396  Drosophila_virilis  A  T  T  G  G  A  A  C  ...    A   
498  DQ471577_110189479  Drosophila_virilis  A  T  T  G  G  A  A  C  ...    A   

    S655 S656 S657 S658 S65

## 3.1 Eliminación de posiciones que presentan homología

Al alinear el conjunto de entrenamiento, podemos percatarnos de que ciertas posiciones se encuentran completamente conservadas (el mismo nucleótido aparece en la misma posición para todas las secuencias). Estos sitios homólogos pueden ser identificados, a partir de lo cual podemos eliminar las columnas que corresponden a dichos sitios de homología tanto en el conjunto de entrenamiento como en el de prueba. Esto disminuye la cantidad de datos con las cuales continuaremos trabajando.

In [12]:
"""
Hacemos una iteración a lo largo de cada columna contando los duplicados que encontremos a lo largo de ella.
Para determinar una conservación de nucleótidos del 100%, simplemente comparamos los duplicados con el número
de filas contadas y almacenamos las etiquetas de las columnas donde se cumpla esta condición.

"""

columns = data_train.columns
flag = []

for column in columns:
    s = data_train.duplicated(subset=[column]).sum()
    if s == (len(data_train) - 1):
        flag.append(column) #flag almacena las etiquetas de los sitios de homología


"""
Aquí generamos nuestros dataframes finales donde eliminamos estos sitios de homolgía encontrados en el conjunto de
prueba.

"""
data_train_het = data_train.drop(flag, axis=1)
data_het = data.drop(flag, axis=1)

#print(flag)

Corroboramos que los sitios homólogos hayan sido eliminados y nos quedamos con conjuntos de datos un poco más heterogéneos.

In [13]:
print(data_het)

                     ID S1 S2 S3 S4 S5 S6 S7 S8 S9  ... S645 S646 S648 S651  \
0    DQ471543_110189412  A  T  T  G  G  A  A  C  T  ...    T    T    A    C   
1    DQ471554_110189434  A  T  T  G  G  A  A  C  T  ...    T    C    A    C   
2     DQ383677_87475153  N  N  N  N  N  N  A  C  T  ...    T    C    T    C   
3     DQ383678_87475155  N  N  N  N  N  N  A  C  T  ...    T    C    T    C   
4     DQ383671_87475141  N  N  N  N  N  N  A  C  T  ...    T    C    T    C   
..                  ... .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...   
116            Auxiliar  A  A  A  A  A  A  A  A  A  ...    A    A    A    A   
117            Auxiliar  C  C  C  C  C  C  C  C  C  ...    C    C    C    C   
118            Auxiliar  T  T  T  T  T  T  T  T  T  ...    T    T    T    T   
119            Auxiliar  G  G  G  G  G  G  G  G  G  ...    G    G    G    G   
120            Auxiliar  N  N  N  N  N  N  N  N  N  ...    N    N    N    N   

    S654 S657 S660 S661 S662 S663  
0      A    T  

In [14]:
print(data_train_het)

                     ID              Target S1 S2 S3 S4 S5 S6 S7 S8  ... S645  \
0    DQ471538_110189402    Drosophila_angor  A  T  T  G  G  A  A  C  ...    T   
1    DQ471539_110189404    Drosophila_angor  A  T  C  G  G  A  A  C  ...    T   
2    DQ471540_110189406    Drosophila_angor  A  T  T  G  G  A  A  C  ...    T   
3    DQ471541_110189408    Drosophila_angor  A  T  C  G  G  T  A  C  ...    T   
4    DQ471542_110189410    Drosophila_angor  A  T  T  G  G  A  A  C  ...    T   
..                  ...                 ... .. .. .. .. .. .. .. ..  ...  ...   
494   DQ426805_90018989  Drosophila_virilis  N  N  N  G  G  A  A  C  ...    T   
495   DQ426806_90018991  Drosophila_virilis  N  N  N  G  G  A  A  C  ...    T   
496   DQ426807_90018993  Drosophila_virilis  N  N  N  G  G  A  A  C  ...    T   
497  DQ471535_110189396  Drosophila_virilis  A  T  T  G  G  A  A  C  ...    T   
498  DQ471577_110189479  Drosophila_virilis  A  T  T  G  G  A  A  C  ...    T   

    S646 S648 S651 S654 S65

**NOTA:** Si se analiza el código escrito hasta este punto, se observará que todas las variables se diseñaron con la intención de poder trabajar con cualquier par de archivos FASTA de entrenamiento y prueba que sean proporcionados.

# 4. Implementando el modelo de aprendizaje supervisado

## 4.1 Randomforest classifier
Aquí implementamos el clasificador con el que se contaba previamente a partir del DEMO proporcionado. Podemos comparar los resultados obtenidos con el procesamiento de datos previamente realizado.

In [15]:
from sklearn import datasets, model_selection, metrics, ensemble

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import cohen_kappa_score

### 4.1.1. Exploración de datos:

In [16]:
Y_train = data_train_het['Target']
Y_train

0        Drosophila_angor
1        Drosophila_angor
2        Drosophila_angor
3        Drosophila_angor
4        Drosophila_angor
              ...        
494    Drosophila_virilis
495    Drosophila_virilis
496    Drosophila_virilis
497    Drosophila_virilis
498    Drosophila_virilis
Name: Target, Length: 499, dtype: object

In [17]:
X_train = data_train_het.loc[:, 'S1':'S663']
X_train.head

<bound method NDFrame.head of     S1 S2 S3 S4 S5 S6 S7 S8 S9 S10  ... S645 S646 S648 S651 S654 S657 S660  \
0    A  T  T  G  G  A  A  C  T   T  ...    T    T    A    C    A    T    A   
1    A  T  C  G  G  A  A  C  T   T  ...    T    T    A    T    A    T    A   
2    A  T  T  G  G  A  A  C  T   T  ...    T    T    A    C    A    T    A   
3    A  T  C  G  G  T  A  C  A   C  ...    T    C    T    T    A    T    A   
4    A  T  T  G  G  A  A  C  T   T  ...    T    T    A    C    A    T    A   
..  .. .. .. .. .. .. .. .. ..  ..  ...  ...  ...  ...  ...  ...  ...  ...   
494  N  N  N  G  G  A  A  C  A   T  ...    T    T    A    C    A    T    A   
495  N  N  N  G  G  A  A  C  A   T  ...    T    T    A    C    A    T    A   
496  N  N  N  G  G  A  A  C  A   T  ...    T    T    A    C    A    T    A   
497  A  T  T  G  G  A  A  C  A   T  ...    T    T    A    C    A    T    A   
498  A  T  T  G  G  A  A  C  A   T  ...    T    T    A    C    A    T    A   

    S661 S662 S663  
0      G    

### 4.1.2. *One-hot encoding* del conjunto de entrenamiento

In [18]:
# list(data) or
data_train_het.columns

Index(['ID', 'Target', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8',
       ...
       'S645', 'S646', 'S648', 'S651', 'S654', 'S657', 'S660', 'S661', 'S662',
       'S663'],
      dtype='object', length=268)

In [19]:
df = data_train_het.loc[:, 'Target':'S663']
df.columns

Index(['Target', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9',
       ...
       'S645', 'S646', 'S648', 'S651', 'S654', 'S657', 'S660', 'S661', 'S662',
       'S663'],
      dtype='object', length=267)

In [20]:
df_X = data_train_het.loc[:, 'S1':'S663']
df_X.columns

Index(['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
       ...
       'S645', 'S646', 'S648', 'S651', 'S654', 'S657', 'S660', 'S661', 'S662',
       'S663'],
      dtype='object', length=266)

In [21]:
one_hot_encoded_data = pd.get_dummies(df, columns = df_X.columns, dtype=int)
print(one_hot_encoded_data)

                 Target  S1_A  S1_N  S2_N  S2_T  S3_C  S3_N  S3_T  S4_G  S4_N  \
0      Drosophila_angor     1     0     0     1     0     0     1     1     0   
1      Drosophila_angor     1     0     0     1     1     0     0     1     0   
2      Drosophila_angor     1     0     0     1     0     0     1     1     0   
3      Drosophila_angor     1     0     0     1     1     0     0     1     0   
4      Drosophila_angor     1     0     0     1     0     0     1     1     0   
..                  ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
494  Drosophila_virilis     0     1     1     0     0     1     0     1     0   
495  Drosophila_virilis     0     1     1     0     0     1     0     1     0   
496  Drosophila_virilis     0     1     1     0     0     1     0     1     0   
497  Drosophila_virilis     1     0     0     1     0     0     1     1     0   
498  Drosophila_virilis     1     0     0     1     0     0     1     1     0   

     ...  S657_T  S660_A  S

Eliminamos las filas auxiliares:

In [22]:
one_hot_encoded_data = one_hot_encoded_data.iloc[:-1, :]
one_hot_encoded_data = one_hot_encoded_data.iloc[:-1, :]
one_hot_encoded_data = one_hot_encoded_data.iloc[:-1, :]
one_hot_encoded_data = one_hot_encoded_data.iloc[:-1, :]
one_hot_encoded_data = one_hot_encoded_data.iloc[:-1, :]
print(one_hot_encoded_data)

                     Target  S1_A  S1_N  S2_N  S2_T  S3_C  S3_N  S3_T  S4_G  \
0          Drosophila_angor     1     0     0     1     0     0     1     1   
1          Drosophila_angor     1     0     0     1     1     0     0     1   
2          Drosophila_angor     1     0     0     1     0     0     1     1   
3          Drosophila_angor     1     0     0     1     1     0     0     1   
4          Drosophila_angor     1     0     0     1     0     0     1     1   
..                      ...   ...   ...   ...   ...   ...   ...   ...   ...   
489  Drosophila_subquinaria     0     1     1     0     0     1     0     0   
490      Drosophila_virilis     1     0     0     1     0     0     1     1   
491      Drosophila_virilis     0     1     1     0     0     1     0     1   
492      Drosophila_virilis     0     1     1     0     0     1     0     1   
493      Drosophila_virilis     0     1     1     0     0     1     0     1   

     S4_N  ...  S657_T  S660_A  S660_G  S661_G  S66

In [23]:
le = preprocessing.LabelEncoder()
le.fit(one_hot_encoded_data.Target)
list(le.classes_)
one_hot_encoded_data['Target'] = le.transform(one_hot_encoded_data.Target)
print(one_hot_encoded_data)

     Target  S1_A  S1_N  S2_N  S2_T  S3_C  S3_N  S3_T  S4_G  S4_N  ...  \
0         0     1     0     0     1     0     0     1     1     0  ...   
1         0     1     0     0     1     1     0     0     1     0  ...   
2         0     1     0     0     1     0     0     1     1     0  ...   
3         0     1     0     0     1     1     0     0     1     0  ...   
4         0     1     0     0     1     0     0     1     1     0  ...   
..      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
489      17     0     1     1     0     0     1     0     0     1  ...   
490      18     1     0     0     1     0     0     1     1     0  ...   
491      18     0     1     1     0     0     1     0     1     0  ...   
492      18     0     1     1     0     0     1     0     1     0  ...   
493      18     0     1     1     0     0     1     0     1     0  ...   

     S657_T  S660_A  S660_G  S661_G  S661_T  S662_G  S662_T  S663_C  S663_G  \
0         1       1       0     

In [24]:
le.inverse_transform(one_hot_encoded_data['Target'])

array(['Drosophila_angor', 'Drosophila_angor', 'Drosophila_angor',
       'Drosophila_angor', 'Drosophila_angor', 'Drosophila_angor',
       'Drosophila_angor', 'Drosophila_angor', 'Drosophila_angor',
       'Drosophila_angor', 'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_barutani', 'Drosophila_barutani',
       'Drosophila_barutani', 'Drosophila_barutani',
       'Drosophila_barutani', 'Drosophila_beppui', 'Drosophila_beppui',
       'Drosophila_beppui', 'Drosophila_daruma', 'Drosophila_daruma',
       'Drosophila_daruma', 'Drosophila_daruma', 'Drosophila_falleni',
       'Drosophila_falleni', 'Drosophila_falleni', 'Drosophila_falleni',
       'Drosophi

In [25]:
target = one_hot_encoded_data['Target']
one_hot_encoded_data_x = one_hot_encoded_data.loc[:, 'S1_A':'S663_T']
x_train, x_test, y_train, y_test = model_selection.train_test_split(one_hot_encoded_data_x, target, test_size = 0.3, random_state =0)
print("Training Dataset:", x_train.shape)
print("Test Dataset:", x_test.shape)

Training Dataset: (345, 686)
Test Dataset: (149, 686)


### 4.1.3. Entrenamiento del modelo

In [26]:
classifier = ensemble.RandomForestClassifier()
model = classifier.fit(x_train,y_train)
print(model)

RandomForestClassifier()


In [27]:
print(x_test)

     S1_A  S1_N  S2_N  S2_T  S3_C  S3_N  S3_T  S4_G  S4_N  S5_C  ...  S657_T  \
339     0     1     1     0     0     1     0     0     1     0  ...       1   
96      0     1     1     0     0     1     0     0     1     0  ...       1   
326     0     1     1     0     0     1     0     0     1     0  ...       1   
245     0     1     1     0     0     1     0     0     1     0  ...       1   
76      1     0     0     1     0     0     1     1     0     0  ...       1   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...     ...   
4       1     0     0     1     0     0     1     1     0     0  ...       1   
418     0     1     1     0     0     1     0     0     1     0  ...       1   
175     0     1     1     0     0     1     0     0     1     0  ...       1   
453     0     1     1     0     0     1     0     0     1     0  ...       1   
5       1     0     0     1     0     0     1     1     0     0  ...       1   

     S660_A  S660_G  S661_G  S661_T  S6

In [28]:
model.score(x_test,y_test)

0.9798657718120806

Comparando con la demo, el incremento en el score ronda del 0.6%. Sin embargo, se está trabajando aproximadamente con un 20% de los datos originales.

![model-predict-colab.PNG](https://drive.google.com/uc?export=view&id=1nE29f1r9ugqPANMTWoO5dBfxQjo0Przz)

In [29]:
y_pred = model.predict(x_test)
report = metrics.classification_report(y_test,y_pred)
print(report)
print(y_pred)

              precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         1
           5       0.75      1.00      0.86         3
           6       1.00      1.00      1.00         9
           7       1.00      0.50      0.67         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00         8
          11       1.00      1.00      1.00         9
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00        24
          15       0.96      1.00      0.98        26
          16       0.90      1.00      0.95         9
          17       1.00      0.97      0.99        38
          18       1.00      1.00      1.00         2

    accuracy              

In [30]:
cohen_kappa_score(y_test,y_pred)

0.9766140002092707

Aquí también podemos observar esta diferencia mínima obtenida para *cohen_kappa_score*:

![cohen.PNG](https://drive.google.com/uc?export=view&id=1Javqh3SV8yBpXeEtzsHf8ffPq20iFpzD)


In [31]:
le.inverse_transform(y_pred)

array(['Drosophila_recens', 'Drosophila_mettleri', 'Drosophila_recens',
       'Drosophila_pachea', 'Drosophila_melanogaster',
       'Drosophila_arizonae', 'Drosophila_mojavensis',
       'Drosophila_recens', 'Drosophila_simulans', 'Drosophila_simulans',
       'Drosophila_recens', 'Drosophila_mettleri', 'Drosophila_recens',
       'Drosophila_recens', 'Drosophila_montana', 'Drosophila_recens',
       'Drosophila_pachea', 'Drosophila_virilis', 'Drosophila_recens',
       'Drosophila_falleni', 'Drosophila_mettleri', 'Drosophila_simulans',
       'Drosophila_subquinaria', 'Drosophila_melanogaster',
       'Drosophila_falleni', 'Drosophila_recens', 'Drosophila_arizonae',
       'Drosophila_recens', 'Drosophila_recens', 'Drosophila_pachea',
       'Drosophila_pachea', 'Drosophila_subquinaria',
       'Drosophila_virilis', 'Drosophila_subquinaria',
       'Drosophila_montana', 'Drosophila_subquinaria',
       'Drosophila_mojavensis', 'Drosophila_subquinaria',
       'Drosophila_pachea', 'D

In [32]:
le.inverse_transform(y_test)

array(['Drosophila_recens', 'Drosophila_mettleri', 'Drosophila_recens',
       'Drosophila_pachea', 'Drosophila_melanogaster',
       'Drosophila_arizonae', 'Drosophila_mojavensis',
       'Drosophila_recens', 'Drosophila_simulans', 'Drosophila_simulans',
       'Drosophila_recens', 'Drosophila_mettleri', 'Drosophila_recens',
       'Drosophila_recens', 'Drosophila_montana', 'Drosophila_recens',
       'Drosophila_pachea', 'Drosophila_virilis', 'Drosophila_recens',
       'Drosophila_falleni', 'Drosophila_mettleri', 'Drosophila_simulans',
       'Drosophila_subquinaria', 'Drosophila_melanogaster',
       'Drosophila_falleni', 'Drosophila_recens', 'Drosophila_arizonae',
       'Drosophila_recens', 'Drosophila_subquinaria', 'Drosophila_pachea',
       'Drosophila_pachea', 'Drosophila_subquinaria',
       'Drosophila_virilis', 'Drosophila_subquinaria',
       'Drosophila_montana', 'Drosophila_subquinaria',
       'Drosophila_mojavensis', 'Drosophila_subquinaria',
       'Drosophila_pachea

In [33]:
pred_df = pd.DataFrame()
pred_df['Esp'] = le.inverse_transform(y_pred)
pred_df['Obs'] = le.inverse_transform(y_test)
print(pred_df)

                         Esp                      Obs
0          Drosophila_recens        Drosophila_recens
1        Drosophila_mettleri      Drosophila_mettleri
2          Drosophila_recens        Drosophila_recens
3          Drosophila_pachea        Drosophila_pachea
4    Drosophila_melanogaster  Drosophila_melanogaster
..                       ...                      ...
144         Drosophila_angor         Drosophila_angor
145   Drosophila_subquinaria   Drosophila_subquinaria
146       Drosophila_navojoa       Drosophila_navojoa
147   Drosophila_subquinaria   Drosophila_subquinaria
148       Drosophila_falleni         Drosophila_angor

[149 rows x 2 columns]


## 4.2 Evaluando el modelo

### 4.2.1 One-hot encoding del conjunto de prueba

In [34]:
data_het.head

<bound method NDFrame.head of                      ID S1 S2 S3 S4 S5 S6 S7 S8 S9  ... S645 S646 S648 S651  \
0    DQ471543_110189412  A  T  T  G  G  A  A  C  T  ...    T    T    A    C   
1    DQ471554_110189434  A  T  T  G  G  A  A  C  T  ...    T    C    A    C   
2     DQ383677_87475153  N  N  N  N  N  N  A  C  T  ...    T    C    T    C   
3     DQ383678_87475155  N  N  N  N  N  N  A  C  T  ...    T    C    T    C   
4     DQ383671_87475141  N  N  N  N  N  N  A  C  T  ...    T    C    T    C   
..                  ... .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...   
116            Auxiliar  A  A  A  A  A  A  A  A  A  ...    A    A    A    A   
117            Auxiliar  C  C  C  C  C  C  C  C  C  ...    C    C    C    C   
118            Auxiliar  T  T  T  T  T  T  T  T  T  ...    T    T    T    T   
119            Auxiliar  G  G  G  G  G  G  G  G  G  ...    G    G    G    G   
120            Auxiliar  N  N  N  N  N  N  N  N  N  ...    N    N    N    N   

    S654 S657 S660 S6

In [35]:
df_X_test = data_het.loc[:, 'S1':'S663']
df_X_test.columns

Index(['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
       ...
       'S645', 'S646', 'S648', 'S651', 'S654', 'S657', 'S660', 'S661', 'S662',
       'S663'],
      dtype='object', length=266)

In [36]:
one_hot_encoded_test = pd.get_dummies(df_X_test, columns = df_X_test.columns, dtype = int)
print(one_hot_encoded_test)

     S1_A  S1_C  S1_G  S1_N  S1_T  S2_A  S2_C  S2_G  S2_N  S2_T  ...  S662_A  \
0       1     0     0     0     0     0     0     0     0     1  ...       0   
1       1     0     0     0     0     0     0     0     0     1  ...       0   
2       0     0     0     1     0     0     0     0     1     0  ...       0   
3       0     0     0     1     0     0     0     0     1     0  ...       0   
4       0     0     0     1     0     0     0     0     1     0  ...       0   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...     ...   
116     1     0     0     0     0     1     0     0     0     0  ...       1   
117     0     1     0     0     0     0     1     0     0     0  ...       0   
118     0     0     0     0     1     0     0     0     0     1  ...       0   
119     0     0     1     0     0     0     0     1     0     0  ...       0   
120     0     0     0     1     0     0     0     0     1     0  ...       0   

     S662_C  S662_G  S662_N  S662_T  S6

In [37]:
one_hot_encoded_test = one_hot_encoded_test.iloc[:-1, :]
one_hot_encoded_test = one_hot_encoded_test.iloc[:-1, :]
one_hot_encoded_test = one_hot_encoded_test.iloc[:-1, :]
one_hot_encoded_test = one_hot_encoded_test.iloc[:-1, :]
one_hot_encoded_test = one_hot_encoded_test.iloc[:-1, :]
print(one_hot_encoded_test)


     S1_A  S1_C  S1_G  S1_N  S1_T  S2_A  S2_C  S2_G  S2_N  S2_T  ...  S662_A  \
0       1     0     0     0     0     0     0     0     0     1  ...       0   
1       1     0     0     0     0     0     0     0     0     1  ...       0   
2       0     0     0     1     0     0     0     0     1     0  ...       0   
3       0     0     0     1     0     0     0     0     1     0  ...       0   
4       0     0     0     1     0     0     0     0     1     0  ...       0   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...     ...   
111     1     0     0     0     0     0     0     0     0     1  ...       0   
112     1     0     0     0     0     0     0     0     0     1  ...       0   
113     0     0     0     1     0     0     0     0     1     0  ...       0   
114     0     0     0     1     0     0     0     0     1     0  ...       0   
115     0     0     0     1     0     0     0     0     1     0  ...       0   

     S662_C  S662_G  S662_N  S662_T  S6

In [38]:
one_hot_encoded_test = one_hot_encoded_test[x_train.columns]

In [39]:
pred = model.predict(one_hot_encoded_test)
le.inverse_transform(pred)

array(['Drosophila_angor', 'Drosophila_angor', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_barutani', 'Drosophila_falleni', 'Drosophila_falleni',
       'Drosophila_falleni', 'Drosophila_innubila', 'Drosophila_innubila',
       'Drosophila_innubila', 'Drosophila_innubila',
       'Drosophila_melanogaster', 'Drosophila_melanogaster',
       'Drosophila_mettleri', 'Drosophila_mettleri',
       'Drosophila_mettleri', 'Drosophila_mettleri',
       'Drosophila_mettleri', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_montana', 'Drosophila_montana', 'Drosophila_montana',
       'Drosophila_montana', 'Drosophila_montana', 'Drosophila_montana',
       'Drosophila_montana', 'Drosophila_montana',
       'Drosophila_nigrospir

In [40]:
print(data.shape)

(121, 664)


In [41]:
df_test = data.iloc[:-1, :]
df_test = df_test.iloc[:-1, :]
df_test = df_test.iloc[:-1, :]
df_test =df_test.iloc[:-1, :]
df_test = df_test.iloc[:-1, :]

In [42]:
df_test['prediccion_spp'] = le.inverse_transform(pred)
df_test['prediccion_etiqueta'] = pred
print(df_test[['ID','prediccion_spp','prediccion_etiqueta']])

                     ID          prediccion_spp  prediccion_etiqueta
0    DQ471543_110189412        Drosophila_angor                    0
1    DQ471554_110189434        Drosophila_angor                    0
2     DQ383677_87475153     Drosophila_arizonae                    1
3     DQ383678_87475155     Drosophila_arizonae                    1
4     DQ383671_87475141     Drosophila_arizonae                    1
..                  ...                     ...                  ...
111   AY154408_25989948  Drosophila_subquinaria                   17
112   AY154424_25989980  Drosophila_subquinaria                   17
113  DQ851620_114187041  Drosophila_subquinaria                   17
114   DQ426803_90018985      Drosophila_virilis                   18
115   DQ426800_90018979      Drosophila_virilis                   18

[116 rows x 3 columns]


Almacenamos los datos de la predicción en un .csv:

In [43]:
predict = df_test[['ID','prediccion_spp','prediccion_etiqueta']]
predict.to_csv('Kamikaze.csv', index=False)